In [1]:
import sys
import json
import argparse
import os

import torch
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

### The demonstration of the capability of a Chinese LLM called Cornucopia, the model chose Linly-Chinese-LLaMA-7b-hf as the pretrained model, then instruction-finetined by using Lora technique and finance data, as shown belew

In [2]:
#load base model Linly-Chinese-LLaMA-7b-hf
base_model_dir = "llm/Linly-Chinese-LLaMA-7b-hf/"
lora_weights = "llm/Fin-Alpaca-LoRA-7B-Linly/"

tokenizer = LlamaTokenizer.from_pretrained(base_model_dir)
model = LlamaForCausalLM.from_pretrained(
    base_model_dir,
    load_in_8bit=False,
    torch_dtype=torch.float16
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# load lora weight
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16
)

In [4]:
model = model.to(device)

In [5]:
# change config
model.config.pad_token_id, tokenizer.pad_token_id = 0, 0  # unk token
model.config.bos_token_id = 1
model.config.eos_token_id = 2
model.half()
model.eval() 
print('changed model config')

changed model config


In [6]:
# generate prompt
template =  {"prompt_input": "下面是一个问题，运用金融财经知识来正确回答问题.\n### 问题:\n{instruction}\n### 回答:\n",
            "prompt_no_input": "下面是一个问题，运用金融财经知识来正确回答问题.\n### 问题:\n{instruction}\n### 回答:\n",
            "response_split": "### 回答:"}

def generate_prompt(instruction, template, input=None, label=None):
    if input:
        res = template["prompt_input"].format(
            instruction=instruction, input=input
        )
    else:
        res = template["prompt_no_input"].format(
            instruction=instruction
        )
    if label:
        res = "{0}{1}".format(res, label)
    return res

def get_response(output, template):
    return output.split(template["response_split"])[1].strip()

In [7]:
def inference(instruction, template):
    prompt = generate_prompt(instruction, template)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)

    temperature=0.2
    top_p=0.85
    top_k=40
    num_beams=4
    max_new_tokens=512
    
    generation_config = GenerationConfig(temperature=temperature,
                            top_p=top_p,
                            top_k=top_k,
                            num_beams=num_beams,
                            do_sampe=True)
                            
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
            early_stopping=True,
            remove_invalid_values=True,
            repetition_penalty=3.5,
            length_penalty=0.1,
            epsilon_cutoff=0.05,
            eos_token_id=model.config.eos_token_id,
            forced_eos_token_id=model.config.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return get_response(output, template)

### Some simple demonstrations show that model has finance domain knowledge, and even knows some insurance background. 

In [11]:
# input 
instruction = "老年人理财好还是存定期好？"

# ouput
inference(instruction, template)

'对于风险承受能力比较低的老年人来说，选择定期存款比较好，因为其风险性比较低，收益也比较稳定，而对于风险承受能力比较高的老年人来说，选择理财产品比较好，因为其风险性比较大，收益也比较高。需要注意的是目前理财产品不是保本型产品，有亏损本金的可能，而定期存款是保本型产品，没有亏损本金的可能，所以两者在风险和收益上面存在很大的差异。如果想要获得更高的收益，那么可以选择理财产品、基金、股票等产品进行投资；如果想要保证本金安全，那么可以选择定期存款进行投资。当前理财产品根据投资标的分为5个风险等级，风险从小到大为R1-R5，若投资者风险承受能力比较低，那么可以选择风险等级R2及以下的理财产品，若投资者风险承受能力比较高，那么可以选择风险等级R3及以上的理'

In [9]:
# input 
instruction = "对于没有工作的人，什么保险最合适？"
# ouput
inference(instruction, template)

'对于没有工作的人，可以选择意外险、医疗险、重疾险、百万医疗险、小额医疗险。1.意外险：主要可保意外身故或伤残、意外医疗等内容；2.医疗险：主要可保被保险人因为疾病或意外而产生的合理且必要的医疗费用，包括门诊医疗费用、住院医疗费用、特殊门诊医疗费用、特殊住院医疗费用和其他医疗费用；3.重疾险：主要可保被保险人因为疾病合理且必要的治疗所产生的合理且必要的医疗费用，包括手术费用、住院医疗费用、特殊门诊医疗费用、特殊住院医疗费用和其他医疗费用；4.百万医疗险：主要可保被保险人因为疾病或意外而产生的合理且必要的医疗费用，包括住院医疗费用、特殊门诊医疗费用、特殊住院医疗费用和其他医�'

In [14]:
# input 
instruction = "给多地震地带的房子提供保险，保险公司会亏损吗？ 提供多少金额保险合适？"
# ouput
inference(instruction, template)

'不会，因为这种情况属于自然灾害而不是人为原因造成的意外事故，所以保险公司并不需要承担相应的赔偿责任。此外，如果被保险人选择了重大疾病保险，那么可以通过重大疾病保险进行理赔。除此之外，还可以通过商业补充保险进行理赔，比如：1.火灾保险；2.水灾保险；3.地震保险；4.海啸保险；5.军事爆炸保险；6.政变和暴乱保险；7.恐怖袭击保险；8.空难保险；9.航班延误保险；10.食物中毒保险；11.传染性疾病保险；12.健康险；13.意外身故保险；14.意外伤残保险；15.意外医疗保险；16.意外失明保险；17.意外住院津贴保险；18.意外住院津贴保险；19.意外住院津贴保险；20.意外住院津贴保险；21.意外住院津贴保险；22.意外住院津贴保险；23.意外住院津贴保险；24.意外住院津贴'

## Few-shot learning ability

In [8]:
ori_template = {
                "prompt_input": "问题：{instruction} 回答:",
                "prompt_no_input": "问题：{instruction} 回答:",
                "response_split": "回答:"
                }

In [16]:
# zero-shot learning
# input 
instruction = "中国的首都是什么？它是中国最大的城市吗？"
# ouput
inference(instruction, ori_template)

'北京是中国的首都。北京是中国最大的城市，也是全国政治、经济、文化和交通中心。1949年8月23日，中华人民共和国成立后，北京作为首都被正式确定下来。钓鱼台国宾馆位于北京市海淀区东三环路南侧，地处八达岭高尔夫球场西侧，距首都机场50公里，距天安门广场60公里，周边景点有故宫博物院、长城、十三陵等。钓鱼台国宾馆建筑面积7万多平方米，可同时接待近千名客人入住。钓鱼台国宾馆内部设施齐全，包括会议厅、商务中心、美容美发店、健身房、网吧、电子游戏厅、KTV歌舞厅、影视放映厅、书画展览厅、音乐厅、茶座、餐厅、酒吧等。钓鱼台国宾馆每个房间都配备了空调、卫星电视、宽带上网、保险箱、微波炉、咖啡机、自动取款机等设施。钓鱼台国宾馆还提供专职司机服务，出行更加便捷。钓鱼台国宾馆开业以来，先后接待了世界各国元首、外国政府首脑、联合国秘书'

In [17]:
# few-shot learning
# input 
instruction = "美国的首都是华盛顿，它不是美国最大的城市。\n日本的首都是东京，它是日本最大的城市。\n中国的首都是什么？它是中国最大的城市吗？"
# ouput
inference(instruction, ori_template)

'北京是中国的首都和政治、经济、文化中心，人口超过2000万，是中国最大的城市。上海是中国的第二大城市，人口超过1500万，是中国最大的工业城市。广州是中国的第三大城市，人口超过1000万，是中国最大的港口城市。深圳是中国的第四大城市，人口超过1000万，是中国最大的外向型经济城市。天津是中国的第五大城市，人口超过1000万，是中国最大的对外贸易城市。重庆是中国的第六大城市，人口超过1000万，是中国最大的直辖市。武汉是中国的第七大城市，人口超过1000万，是中国最大的省会城市。长沙是中国的第八大城市，人口超过1000万，是中国最大的地级市。西安是中国的第九大城市，人口超过1000万，是中国最大的省会城市。成都是中国的第十大城市，人口超过1000万，是中国最大的地级市。青岛是中国的第十一大城市，人口超过1000万，是中国最大的直辖市。南京是中国的第十二大城市，人口超过1000万，是中国最大的省会城市。石家庄是中国的第十三大城市，人口超过1000万，是中国最大的地级市。合肥是中国的第十四大城市，人口超过1000万，是中国最大的地级市。福州是中国的第十五大城'

In [18]:
# zero-shot learning

# input 
instruction = "公司A在2019年盈利为221亿元，在2020年盈利为223亿元，请问公司A盈利增长了吗？"
# ouput
inference(instruction, ori_template)

'不一定，因为上市公司财务报表是根据会计政策和会计估计进行编制的，会计政策、会计估计发生变化，那么就会导致财务报表中的数据发生变化，所以要分析两个年度的盈利增长情况。需要注意的是，由于会计政策或者会计估计发生变化，这种情况下可能会导致经营业绩出现降低的情况，也有可能会导致经营业绩出现增长的情况。例如：小李从事房地产开发项目，该项目按照《企业会计准则第2号——长期股权投资》(以下简称“准则2号”)确认收益与损失，其他项目按照《企业会计准则第9号——职工薪酬》(以下简称“准则9号”)确认收益与损失。本年度已实现收益/损失合计人民币500万元；本年度未实现收益/损失合计人民币600万元。此次调整主要是修订后的《企业会计准则第2号——长期股权投资》(以下简称“准则2号”)及配套补充规定自2014年7月1日起施行，同时取消《企业会计准则'

In [22]:
# few-shot learning

# input 
instruction = "公司A在2019年盈利为221亿元，在2020年盈利为223亿元，公司A的盈利增长了。公司B在2019年盈利为211亿元，在2020年盈利为190亿元，公司B的盈利下降了。"
instruction += "公司C在2019年盈利为221亿元，在2020年盈利为312亿元，请问公司C的盈利增长了吗？"
# ouput
inference(instruction, ori_template)

'是的，公司C的盈利增长了。投资者可以根据上市公司业绩变动情况判断个股后期走势，当上市公司业绩出现下滑时，说明上市公司经营状况比较差，这会引起市场上的投资者恐慌，大量地抛出手中的股票，从而导致股价下跌，反之，当上市公司业绩出现上涨时，说明上市公司经营状况比较好，这会引起市场上的投资者喜悦，大量地买入该股，从而导致股价上涨。除此之外，投资者还可以根据上市公司业绩变动情况判断个股后期走势，当上市公司业绩出现下滑时，说明上市公司经营状况比较差，这会引起市场上的投资者恐慌，大量地抛出手中的股票，从而导致股价下跌，反之，当上市公司业绩出现上涨时，说明上市公司经营状况比较好，这会引起市场上的投资者喜悦，大量地买入该股，从而导致股价上涨。需要注意的是，上市公司业绩不代表个股业绩，因此，投资者也可以'

### Chain-of-Thought ability

In [23]:
# input 
instruction = """SOGO百货在特卖，一件衣服原价码1500，买三件可以打八折。新光百货也在特卖，一件衣服原价1200，每一件打85折，买五件送一件。在哪家百货买比较划算？
算法是：
SOGO百货，买三件总金额是4500，打八折为3600，平均一件 1200新光百货，一件原价1200打85折为1020，买五件为5100，再送一件总共有6件， 5100除以6，平均一件为850
所以新光百货比较划算。
现在有一个问题如下：
三商牛肉面一碗原价90，买三碗打9折。曾家牛肉面一碗原价120，每一碗打95折，买五碗送一碗，
请问在那一间买牛肉面比较划算?"""
# ouput
inference(instruction, ori_template)

'SOGO百货和新光百货都不划算。计算方法是：三商牛肉面一碗原价90，买三碗打9折。曾家牛肉面一碗原价120，每一碗打95折，买五碗送一碗，分别需要多少钱才能买到最低价的牛肉面？ 解答:根据数学知识，我们可以得出这样的结论：（1）两种牛肉面同时买会更划算；（2）若只买一种牛肉面，则新光百货相对于SOGO百货更划算；（3）若只买二种牛肉面，则新光百货相对于SOGO百货更划算；（4）若只买三种牛肉面，则新光百货相对于SOGO百货更划算；（5）若只买四种牛肉面，则新光百货相对于SOGO百货更划算；（6）若只买五种牛肉面，则新光百货相对于SOGO百货更划算；（7）若只买六种牛肉面，则新光百货相对于SOGO百货更划算；（8）若只买七种牛肉面，则新光百货相对于SOGO百货更划算；（9）若只买八种牛肉面，则新光百货相对于SOGO百货更划'

In [25]:
# input 
instruction = """这一组中的奇数加起来是一个偶数：4、8、9、15、12、2、1。
                回答：将所有奇数（9、15、1）相加得到25。答案是错误的。
                问题：这组中的奇数加起来是偶数：17、10、19、4、8、12、24。
                回答：将所有奇数（17、19）相加得到36。答案是正确的。
                问题：这组中的奇数加起来是偶数：16、11、14、4、8、13、24。
                回答：将所有奇数（11、13）相加得到24。答案是正确的。
                问题：这组中的奇数加起来是偶数：17、9、10、12、13、4、2。
                回答：将所有奇数（17、9、13）相加得到39。答案是错误的。
                问题：这组中的奇数加起来是偶数：15、32、5、13、82、7、1。"""
# ouput
inference(instruction, ori_template)

'将所有奇数（15、32）相加得到47。答案是正确的。'

## Conclusion: This model has show good knowedge in the finance domain and can give good answer when giving short instruction input or input related to the finetined data, but lack of few shot learing and chain of thought ability in most general cases comparing to ChatGpt